# Find a Feasible Mobile Manipulation (from Real Data)

In [1]:
%matplotlib widget

import numpy as np
import feasibility_map as fmap

## 1. Offline

### 1.0. Utils

### 1.1. Configuration

1. `Rsize`: the radius of the robot-base
2. `M`: X-Y-Cr manipulability map

```python
raw:
    [[Cr x y manip],
     [Cr x y manip], ...]
layers:
    [deg]: [[x, ...],
            [y, ...],
            [manip, ...]]
```

In [2]:
Rsize = 0.3

- npy data
    - Cr (radian)
    - x (m)
    - y (m)
    - manip (no unit)

In [9]:
# In this file, Cr is radians.
M = np.load('../config/socialrobot_right_z75_2021-01-25.npy')
# So, convert it to degrees.
M[:,0] = np.round(np.degrees(M[:,0]), decimals=3)
print(M.shape)
print(M)

(2795, 4)
[[-6.00000000e+01  3.09000000e-01 -3.00000000e-01  6.91546727e-02]
 [-4.50000000e+01  3.09000000e-01 -3.00000000e-01  7.27504883e-02]
 [-3.00000000e+01  3.09000000e-01 -3.00000000e-01  7.75980486e-02]
 ...
 [ 7.50000000e+01  1.59000000e-01  5.00000000e-02  5.84465707e-02]
 [ 9.00000000e+01  1.59000000e-01  5.00000000e-02  6.61140770e-02]
 [ 9.00000000e+01  1.59000000e-01  1.00000000e-01  5.97837489e-02]]


### 1.2. Preparation

1. Convert the `M` to a feasibility map `F` (helical shape).

Cr, x, y, manipulability

In [10]:
F = fmap.FeasibilityMap(M, Rsize, is_jupyter=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 230)
(3, 230)
(2795, 4)


## 2. Online

### 2.1. Input

Known
1. `self.feasi_layers`: feasibility map
2. `self.robot_radius`: robot base (circle radius)

---
Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-90 ~ 90) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-180 ~ 180) => `(min, max)`

In [11]:
from collision import CollisionBox

Pt = (0.0, 0.0)
Obs = [CollisionBox((0.2, 0.0), np.radians(0.0), 0.45, 1.0)]
# Cr = (30, 30)  # min, max
Cr = (0, 0)
Ct = (-np.radians(180), np.radians(180)) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `F` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. And extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`.
5. Remove all obstacle areas from `Fwiped` with the offset of `Rsize`.

In [12]:
F.calc(Pt, Obs, Cr, Ct, (0.05, 0.5, 0.02))

Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
min Cr:  0.0
max Cr:  0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

And extract only the maximum as a `Fmax`.
sq_sections.shape:  (23,)
Fmax.shape:  (15, 4)
Fmax:
 [[ 0.         -0.159       0.1         0.05407761]
 [ 0.         -0.159       0.15        0.06065376]
 [ 0.         -0.209       0.1         0.05544608]
 [ 0.         -0.159       0.2         0.07012463]
 [ 0.         -0.109       0.25        0.07543658]
 [ 0.         -0.059       0.3         0.08000569]
 [ 0.         -0.109       0.3         0.0827823 ]
 [ 0.         -0.159       0.3         0.0826152 ]
 [ 0.         -0.259       0.25        0.09652822]
 [ 0.         -0.359       0.1         0.0900866 ]
 [ 0.         -0.209       0.35        0.09063355]
 [ 0.         -0.359       0.2         0.09336948]
 [ 0.         -0.359       0.25        0.09761725]
 [ 0.         -0.209       0.4         0.09458029]
 [ 0.         -0.459       0.15        0.08853702]]
When Ct == 0.0 deg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -180.0
max Ct:  180.0
Fwiped.shape:  (1582, 5)
Interval is  0.02  [m].


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (1582, 5)
Fclean.shape:  (205, 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.3. Output

1. Candidate poses (sorted in descending order of manipulability)

In [8]:
candidates = F.get_candidates()

num:  1
Fclean[:1]:
 [[ 4.97649627e+01  0.00000000e+00 -3.28258572e-01  1.07453731e-01
   4.87559997e-02]]
Fclean[:1] (sorted):
 [[ 4.97649627e+01  0.00000000e+00 -3.28258572e-01  1.07453731e-01
   4.87559997e-02]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …